In [111]:
import pandas as pd
import numpy as np
import random
import joblib

#### Purpose of Notebook

This notebook was created to create sample datasets to use within a heroku app.New files created are:
1. ratings_demo
2. movies_demo
3. ratings_demo_mx
4. Vectors (U, sigma, Vt) which will be used to calculate dot product in function and user predictions when the function is run.
5. movies_bow_demo

In [112]:
movies = pd.read_csv('Data/movies_sml.csv')
ratings = pd.read_hdf('Data/ratings_hdf.h5')

In [113]:
movies.head()

,movieId,Title,genres,Actors,Director,Plot,Poster
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",John Lasseter,A cowboy doll is profoundly threatened and jea...,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Jumanji,Adventure|Children|Fantasy,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,When two kids find and play a magical board ga...,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Grumpier Old Men,Comedy|Romance,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",Howard Deutch,John and Max resolve to save their beloved bai...,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Waiting to Exhale,Comedy|Drama|Romance,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,"Based on Terry McMillan's novel, this film fol...",https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Father of the Bride Part II,Comedy,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,George Banks must deal not only with the pregn...,https://m.media-amazon.com/images/M/MV5BOTEyNz...


#### Shrink file sizes further for Heroku app

In [114]:
ratings.shape

(1770467, 5)

In [115]:
movies.shape

(9445, 7)

In [116]:
#shrink ratings for demo purposes
ratings_demo = ratings.sample(frac=0.75, random_state=1)

print(ratings_demo.shape)
ratings_demo.head()

(1327850, 5)


,userId,num_user_rated,movieId,Title,rating
71322,117044,148,33493,Star Wars: Episode III - Revenge of the Sith,4.0
755584,49070,127,2997,Being John Malkovich,3.5
1499836,136449,106,2390,Little Voice,5.0
928254,224783,122,597,Pretty Woman,4.0
924584,263937,122,5299,My Big Fat Greek Wedding,4.5


In [117]:
# create new ratings file
ratings_demo.to_hdf('Data/ratings_demo.h5', key='ratings_demo',complib='blosc',complevel=9, mode='w' )

In [118]:
# find only the movies rated by the users in the smaller ratings data set

movies_demo = pd.merge(ratings_demo, movies, how = 'inner', on = 'movieId')

movies_demo.head()

,userId,num_user_rated,movieId,Title_x,rating,Title_y,genres,Actors,Director,Plot,Poster
0,117044,148,33493,Star Wars: Episode III - Revenge of the Sith,4.0,Star Wars: Episode III - Revenge of the Sith,Action|Adventure|Sci-Fi,"Ewan McGregor, Natalie Portman, Hayden Christe...",George Lucas,"Three years into the Clone Wars, the Jedi resc...",https://m.media-amazon.com/images/M/MV5BNTc4MT...
1,169731,123,33493,Star Wars: Episode III - Revenge of the Sith,5.0,Star Wars: Episode III - Revenge of the Sith,Action|Adventure|Sci-Fi,"Ewan McGregor, Natalie Portman, Hayden Christe...",George Lucas,"Three years into the Clone Wars, the Jedi resc...",https://m.media-amazon.com/images/M/MV5BNTc4MT...
2,42574,104,33493,Star Wars: Episode III - Revenge of the Sith,4.0,Star Wars: Episode III - Revenge of the Sith,Action|Adventure|Sci-Fi,"Ewan McGregor, Natalie Portman, Hayden Christe...",George Lucas,"Three years into the Clone Wars, the Jedi resc...",https://m.media-amazon.com/images/M/MV5BNTc4MT...
3,158046,128,33493,Star Wars: Episode III - Revenge of the Sith,3.5,Star Wars: Episode III - Revenge of the Sith,Action|Adventure|Sci-Fi,"Ewan McGregor, Natalie Portman, Hayden Christe...",George Lucas,"Three years into the Clone Wars, the Jedi resc...",https://m.media-amazon.com/images/M/MV5BNTc4MT...
4,97844,108,33493,Star Wars: Episode III - Revenge of the Sith,4.5,Star Wars: Episode III - Revenge of the Sith,Action|Adventure|Sci-Fi,"Ewan McGregor, Natalie Portman, Hayden Christe...",George Lucas,"Three years into the Clone Wars, the Jedi resc...",https://m.media-amazon.com/images/M/MV5BNTc4MT...


In [119]:

movies_demo = movies_demo[['movieId','Title_x','genres','Actors', 'Director', 'Plot', 'Poster' ]]

movies_demo = movies_demo.rename(columns = {'Title_x': 'Title'})

# drop duplicates

movies_demo.drop_duplicates(subset ="movieId", 
                     keep = 'first', inplace = True) 

print(movies_demo.shape)


(8854, 7)


In [120]:
movies_demo.drop_duplicates(subset ="Title", 
                     keep = 'first', inplace = True) 

In [121]:
movies_demo.to_csv('Data/movies_demo.csv', index=False)

In [93]:
movies_demo.head()

,movieId,Title,genres,Actors,Director,Plot,Poster
0,33493,Star Wars: Episode III - Revenge of the Sith,Action|Adventure|Sci-Fi,"Ewan McGregor, Natalie Portman, Hayden Christe...",George Lucas,"Three years into the Clone Wars, the Jedi resc...",https://m.media-amazon.com/images/M/MV5BNTc4MT...
2077,2997,Being John Malkovich,Comedy|Drama|Fantasy,"John Cusack, Cameron Diaz, Ned Bellamy, Eric W...",Spike Jonze,A puppeteer discovers a portal that leads lite...,https://m.media-amazon.com/images/M/MV5BYmUxY2...
5743,2390,Little Voice,Comedy,"Brenda Blethyn, Jane Horrocks, Ewan McGregor, ...",Mark Herman,A shy reclusive lady is convinced by an invisi...,https://m.media-amazon.com/images/M/MV5BN2Y0Yj...
5991,597,Pretty Woman,Comedy|Romance,"Richard Gere, Julia Roberts, Ralph Bellamy, Ja...",Garry Marshall,A man in a legal but hurtful business needs an...,https://m.media-amazon.com/images/M/MV5BNjk2OD...
10033,5299,My Big Fat Greek Wedding,Comedy|Romance,"Nia Vardalos, Michael Constantine, Christina E...",Joel Zwick,A young Greek woman falls in love with a non-G...,https://m.media-amazon.com/images/M/MV5BNzcwOW...


#### Create new Bag of Words csv
New movies_bow csv created based on smaller dataset

In [137]:
movies_bow = movies_demo.copy()

In [138]:
# function to switch all words to lower and remove spaces
def clean_data(x):
    if isinstance(x, list):
        return[str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [139]:
credits = ['Actors', 'Director']

for credit in credits:
    movies_bow[credit] = movies_bow[credit].apply(clean_data)

In [140]:
# remove | from genres

movies_bow['genres'] = movies_bow['genres'].str.replace("|", " ")

In [141]:
#### Combining data 

def create_soup(x):
    return  x['Actors'].replace(',',' ') + ' ' + x['Director'] + ' ' + ' '.join(x['genres'])

In [142]:
movies_bow['bag_of_words'] = movies_bow.apply(create_soup, axis=1)

In [144]:
movies_bow = movies_bow[['movieId','Title','bag_of_words']]
movies_bow.to_csv('Data/movies_bow_demo.csv', index=False)
movies_bow.head()

,movieId,Title,bag_of_words
0,33493,Star Wars: Episode III - Revenge of the Sith,ewanmcgregor natalieportman haydenchristensen ...
2077,2997,Being John Malkovich,johncusack camerondiaz nedbellamy ericweinstei...
5743,2390,Little Voice,brendablethyn janehorrocks ewanmcgregor philip...
5991,597,Pretty Woman,richardgere juliaroberts ralphbellamy jasonale...
10033,5299,My Big Fat Greek Wedding,niavardalos michaelconstantine christinaeleusi...


#### Perform part of the SVD model process
All of the vectors are computed and pickled so that less time will be spent on data processing when the app is being run

In [94]:
# pivot ratings into movie features
ratings_demo_matrix = ratings_demo.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

In [95]:
# create ratings_demo_matrix

ratings_demo_matrix.to_hdf('Data/ratings_demo_mx.h5', key='ratings_demo_matrix',complib='blosc',complevel=9, mode='w' )

In [96]:
# normalize the data by each users mean and convert it from a dataframe to a numpy array.

R = ratings_demo_matrix.values
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [97]:
# Compute the largest k singular values/vectors for a sparse matrix

from scipy.sparse.linalg import svds

U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [98]:
# convert sigma to a diagonal matrix

sigma = np.diag(sigma)

In [99]:
# Pickle of the variables needed to compute the dot product

joblib.dump(U, 'Data/U.pkl')
joblib.dump(Vt, 'Data/Vt.pkl')
joblib.dump(sigma, 'Data/sigma.pkl')

['Data/sigma.pkl']

In [100]:
# calculate the dot product of our vectors
user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [101]:
preds_demo = pd.DataFrame(user_predicted_ratings, columns = ratings_demo_matrix.columns)

preds_demo.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,175727,175731,175975,175991,175995,175999,176001,176003,176157,176211
0,0.134342,0.123447,-0.314752,-0.007506,-0.220334,0.148547,-0.273361,-0.084055,-0.179206,0.230369,...,0.001149,0.001149,0.001827,0.002302,0.002294,0.002294,0.002294,0.002290,0.003442,0.003685
1,0.107459,-0.069435,-0.021670,-0.019094,-0.098950,1.489719,0.233214,-0.039873,0.041659,0.951800,...,-0.001815,-0.001815,0.001766,0.001224,0.001839,0.001839,0.001839,0.002146,0.002824,-0.001393
2,-0.479535,-0.030358,-0.093483,0.047585,-0.111207,0.077514,0.002406,-0.049709,-0.090255,-0.383008,...,0.001362,0.001362,0.002787,0.003657,0.003324,0.003324,0.003324,0.003158,0.005176,-0.001296
3,0.388845,0.912152,-0.121384,-0.014831,-0.264350,2.210059,-0.362664,-0.081191,0.009575,1.523535,...,-0.007588,-0.007588,-0.003405,-0.003026,-0.003357,-0.003357,-0.003357,-0.003523,-0.000141,-0.004896
4,1.448312,0.511455,-0.098407,-0.029741,-0.049380,-0.335932,0.098977,-0.018682,-0.157224,0.782723,...,-0.000292,-0.000292,0.001182,-0.001113,-0.001148,-0.001148,-0.001148,-0.001165,-0.000518,-0.000454


In [107]:
exist_users = ratings_demo['userId'].unique().tolist()

exist_users

[117044,
 49070,
 136449,
 224783,
 263937,
 187855,
 75685,
 106399,
 226832,
 180180,
 253494,
 119916,
 235975,
 25517,
 12269,
 228698,
 164724,
 157442,
 92767,
 80414,
 32948,
 198238,
 163311,
 122803,
 173562,
 140615,
 198462,
 135375,
 73866,
 249913,
 224877,
 252859,
 140931,
 130535,
 252065,
 142616,
 263480,
 250212,
 245929,
 35325,
 152968,
 68801,
 146420,
 256694,
 157661,
 30799,
 23962,
 8624,
 258013,
 31591,
 149449,
 84143,
 46135,
 35721,
 145339,
 78279,
 37051,
 270539,
 186939,
 106967,
 142669,
 63727,
 47358,
 267693,
 239695,
 25879,
 36714,
 155943,
 144454,
 112606,
 20826,
 215907,
 131891,
 167779,
 28707,
 59885,
 179921,
 252005,
 147639,
 270205,
 78966,
 230850,
 27502,
 223348,
 226999,
 225158,
 158282,
 97929,
 140882,
 40353,
 194996,
 19985,
 264072,
 169731,
 121105,
 125790,
 162735,
 210590,
 172401,
 183551,
 59943,
 178076,
 179801,
 114834,
 7787,
 42599,
 133608,
 93046,
 239587,
 134902,
 83089,
 256633,
 124569,
 148787,
 80759,
 257

In [124]:
exist_movies = pd.Series(movies_demo['Title'])

exist_movies

0          Star Wars: Episode III - Revenge of the Sith
2077                               Being John Malkovich
5743                                       Little Voice
5991                                       Pretty Woman
10033                          My Big Fat Greek Wedding
11479                                      Moulin Rouge
12998                                        Phenomenon
14049                                         Get Smart
14229                                         Backdraft
14909                                        Goodfellas
18513                                        Iron Man 3
19140                                          Rounders
19776                                     Stuart Little
20218                                             Ghost
23569                                      Forrest Gump
31573                                             Smoke
32263             The Hunger Games: Mockingjay - Part 1
32711                                           